In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

import random

random.seed(100)
torch.manual_seed(100)
torch.use_deterministic_algorithms(False)

# Limpiar DataBase

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/Python_Archivos/data.csv'
data = pd.read_csv(root, index_col = False)

In [ ]:
data = data.drop('Unnamed: 32', axis = 1)
data = data.drop('id', axis = 1)

In [ ]:
mapDiagnosis = {'M': 0, 'B' : 1}
data['diagnosis'] = data['diagnosis'].map(mapDiagnosis)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    int64  
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

# Base de Datos

In [ ]:
columns = list(data)

x_data = data[columns[1:30]]
y_data = data[columns[0]]

x_data.shape, y_data.shape

((569, 29), (569,))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 20)

In [ ]:
x_train = torch.FloatTensor(x_train.to_numpy())
x_test = torch.FloatTensor(x_test.to_numpy())
y_train = torch.LongTensor(y_train.to_numpy())
y_test = torch.LongTensor(y_test.to_numpy())

# Red Neuronal

Parámetros

In [ ]:
x_train = torch.FloatTensor(x_train.to_numpy())
x_test = torch.FloatTensor(x_test.to_numpy())
y_train = torch.LongTensor(y_train.to_numpy())
y_test = torch.LongTensor(y_test.to_numpy())

In [ ]:
input_size = 30
num_classes = 2
learning_rate = 0.001
num_epochs = 120

In [ ]:
class Model(nn.Module):
  def __init__(self, input_size, num_classes):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(input_size, 120)
    self.fc2 = nn.Linear(120, 60)
    self.fc3 = nn.Linear(60, 30)
    self.fc4 = nn.Linear(30, 2)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.relu(self.fc1(x))
    out = self.relu(self.fc2(out))
    out = self.relu(self.fc3(out))
    out = self.fc4(out)

    return out

In [ ]:
model = Model(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
ls = []
for epoch in range(num_epochs):
  outputs = model(x_train)
  loss = criterion(outputs, y_train)

  loss.backward()
  optimizer.step()
  ls.append(loss.item())


  # print(f'Epoch [{epoch + 1} / {num_epochs}], Loss: {loss.item():.4f}')

plt.plot(ls)

In [ ]:
with torch.no_grad():
  outputs = model(x_test)
  _, predicted = torch.max(outputs.data, 1)
  correct = (predicted == y_test).sum().item()
  total = y_test.size(0)
  print(f'Acuracy: {correct / total:.4}')